# Exploratory analysis

<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/tutorials/02_Exploratory_analysis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

The purpose of exploratory analysis is to understand your data and any idiosyncrasies which may be relevant linking or deduplicating your data.

Splink includes a variety of charts to help with this.

In this notebook we generate some of these charts and interpret the results.


### Read in the data

For the purpose of this tutorial we will use a 1,000 row synthetic dataset that contains duplicates.

The first five rows of this dataset are printed below.

Note that the cluster column represents the 'ground truth' - a column which tells us with which rows refer to the same person. In most real linkage scenarios, we wouldn't have this column (this is what Splink is trying to estimate.)


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev

In [2]:
from splink import splink_datasets, Linker, DuckDBAPI, SettingsCreator
import altair as alt

df = splink_datasets.fake_1000
df = df.drop(columns=["cluster"])
df.head(5)

### Instantiate the linker

Most of Splink's core functionality can be accessed as methods on a linker object. For example, to make predictions, you would call `linker.predict()`.

We therefore begin by instantiating the linker, passing in the data we wish to deduplicate.


In [3]:
# Initialise the linker, passing in the input dataset(s)

db_api = DuckDBAPI()
settings = SettingsCreator(link_type="dedupe_only")

linker = Linker(df, settings, database_api=db_api)

## Analyse missingness


It's important to understand the level of missingness in your data, because columns with higher levels of missingness are less useful for data linking.


In [4]:
from splink.exploratory import completeness_chart
completeness_chart(df, db_api=db_api)

The above summary chart shows that in this dataset, the `email`, `city`, `surname` and `forename` columns contain nulls, but the level of missingness is relatively low (less than 22%).


## Analyse the distribution of values in your data


The distribution of values in your data is important for two main reasons:

1. Columns with higher cardinality (number of distinct values) are usually more useful for data linking. For instance, date of birth is a much stronger linkage variable than gender.

2. The skew of values is important. If you have a `city` column that has 1,000 distinct values, but 75% of them are `London`, this is much less useful for linkage than if the 1,000 values were equally distributed

The `linker.profile_columns()` method creates summary charts to help you understand these aspects of your data.

To profile all columns, leave the column_expressions argument empty.


In [5]:
from splink.exploratory import profile_columns
profile_columns(df, db_api=DuckDBAPI(), top_n=10, bottom_n=5)

This chart is very information-dense, but here are some key takehomes relevant to our linkage:

- There is strong skew in the `city` field with around 20% of the values being `London`. We therefore will probably want to use `term_frequency_adjustments` in our linkage model, so that it can weight a match on London differently to a match on, say, `Norwich`.

- Looking at the "Bottom 5 values by value count", we can see typos in the data in most fields. This tells us this information was possibly entered by hand, or using Optical Character Recognition, giving us an insight into the type of data entry errors we may see.

- Email is a much more uniquely-identifying field than any others, with a maximum value count of 6. It's likely to be a strong linking variable.


!!! note "Further Reading"

    :material-tools: For more on exploratory analysis tools in Splink, please refer to the [Exploratory Analysis API documentation](../../linkerexp.md).

    :bar_chart: For more on the charts used in this tutorial, please refer to the [Charts Gallery](../../charts/index.md#exploratory-analysis).


## Next steps

At this point, we have begun to develop a strong understanding of our data. It's time to move on to estimating a linkage model
